In [2]:
import sqlite3
import pandas as pd
import dateutil.parser
from datetime import datetime
from time import sleep
from random import randint
from bs4 import BeautifulSoup
import requests
from torrequest import TorRequest
from stem import Signal
from stem.control import Controller
import sys

In [3]:
# connect to the news.db, if not exist create one
conn = sqlite3.connect('news.db')
c = conn.cursor()

In [4]:
symbol_list = ['amzn', 'fb', 'goog', 'nflx']

In [4]:
# transform the csv to dataframe
df_amzn = pd.read_csv('./dataset/nasdaq/AMZN_nasdaq_news_merged.csv', index_col=0, header=0)
df_fb = pd.read_csv('./dataset/nasdaq/FB_nasdaq_news_merged.csv', index_col=0, header=0)
df_goog = pd.read_csv('./dataset/nasdaq/GOOG_nasdaq_news_cleaned.csv', index_col=0, header=0)
df_nflx = pd.read_csv('./dataset/nasdaq/NFLX_nasdaq_news_merged.csv', index_col=0, header=0)

In [5]:
# create a new column "unix_timestamp" for the dataframe
df_amzn["unix_timestamp"] = df_amzn["datePublished"].apply(lambda x: dateutil.parser.parse(x).timestamp())
df_fb["unix_timestamp"] = df_fb["datePublished"].apply(lambda x: dateutil.parser.parse(x).timestamp())
df_goog["unix_timestamp"] = df_goog["datePublished"].apply(lambda x: dateutil.parser.parse(x).timestamp())
df_nflx["unix_timestamp"] = df_nflx["datePublished"].apply(lambda x: dateutil.parser.parse(x).timestamp())

In [4]:
# create a table called nasdaq_stock_news
# structure datePublisher, title, text, url, date_EST, data_UTC, unix_timestamp, symbol
try:
    c.execute("CREATE TABLE `nasdaq_stock_news` (`datePublished` TEXT, `title` TEXT, `text` TEXT, `url` TEXT, `date_EST` TEXT, `date_UTC` TEXT, `unix_timestamp` REAL,`symbol` TEXT)")
except:
    print("Error")

Error


In [7]:
# commit the create table action
conn.commit()

In [8]:
# reset the index and reassign the index, coz if the index is not sequence, will return an error during insert
df_amzn = df_amzn.reset_index()
df_fb = df_fb.reset_index()
df_goog = df_goog.reset_index()
df_nflx = df_nflx.reset_index()
df_amzn = df_amzn.drop(labels="index", axis=1)
df_fb = df_fb.drop(labels="index", axis=1)
df_goog = df_goog.drop(labels="index", axis=1)
df_nflx = df_nflx.drop(labels="index", axis=1)

In [5]:
def insert_data_to_sqlite(df, symbol, length):
    
    """
        A function that insert data into the database
    """
    
    for i in range(length):
        c.execute("INSERT INTO nasdaq_stock_news (datePublished, title, text, url, date_EST, date_UTC, unix_timestamp, symbol) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", (df['datePublished'][i], df['title'][i], df['text'][i], df['url'][i], df['date_EST'][i], df['date_UTC'][i], df['unix_timestamp'][i], symbol))
        conn.commit()
        

In [10]:
# 4 dataframe call insert_data_to_sqlite function to insert data
insert_data_to_sqlite(df_amzn, symbol_list[0], df_amzn.shape[0])
insert_data_to_sqlite(df_fb, symbol_list[1], df_fb.shape[0])
insert_data_to_sqlite(df_goog, symbol_list[2], df_goog.shape[0])
insert_data_to_sqlite(df_nflx, symbol_list[3], df_nflx.shape[0])

In [13]:
def check_latest_news(symbol):
    
    """
        return a latest record for finding that symbol lastest news
    """
    
    d = {}
    df_result = pd.read_sql_query("select * from nasdaq_stock_news where symbol = '" + str(symbol) + "'order by datePublished desc limit 1;", conn)
    d['symbol'] = symbol
    d['datePublished'] = df_result['datePublished'].values[0]
    d['title'] = df_result['title'].values[0]
    d['date_UTC'] = df_result['date_UTC'].values[0]
    return d


In [43]:
c = check_latest_news('fb')

In [44]:
print(c['title'])
print(c['symbol'])
print(c['datePublished'])
print(c['date_UTC'])

Stock Market's Muted Reaction to End of Shutdown Shows Primacy of Earnings
fb
2019-01-28T10:52:24-05:00
2019-01-28 15:52:24


In [21]:
def get_connection(links_site):
    
    """
        to get a connection with tor request and try to scrape page
    """
    
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password='16:6A38BF1B14C1F2A660DF93C6DDD45B90B340D7DFF4BBB5AE03FB1F3DE4')
        print("Success!")
        controller.signal(Signal.NEWNYM)
        print("New Tor connection processed")
        tr=TorRequest(password='WTKv36s')
        tr.reset_identity() #Reset Tor
        response= tr.get('http://ipecho.net/plain')
        print("New Ip Address",response.text)
        
        for i in range(10):
            try:
                resp = tr.get(links_site, headers=headers, timeout=5)
                return resp
            except requests.exceptions.Timeout:
                print("Reconnect" + str(i+1))
                with Controller.from_port(port = 9051) as controller:
                    controller.authenticate(password='16:6A38BF1B14C1F2A660DF93C6DDD45B90B340D7DFF4BBB5AE03FB1F3DE4')
                    print("Success!")
                    controller.signal(Signal.NEWNYM)
                    print("New Tor connection processed")
                    tr=TorRequest(password='WTKv36s')
                    tr.reset_identity() #Reset Tor
                    response= tr.get('http://ipecho.net/plain')
                    print("New Ip Address",response.text)
                pass

        return resp

In [22]:
'''Generalized function to get all news-related articles from a Nasdaq webpage'''
def get_news_urls(links_site):
    '''scrape the html of the site'''

    print(links_site)
    
    re = get_connection(links_site)
        
    html = re.content

    '''convert html to BeautifulSoup object'''
    soup = BeautifulSoup(html , 'lxml')


    '''get list of all links on webpage'''
    #links = soup.find_all('a')
    links = soup.find_all(target="_self")


    titles = [title.getText() for title in links]
    titles = titles[1:]

    urls = [link.get('href') for link in links]
    urls = [url for url in urls if url is not None]

    '''Filter the list of urls to just the news articles'''
    news_urls = [url for url in urls if '/article/' in url]
    
    return titles, news_urls

In [23]:
def check_exist(db_title ,titles_list):
    
    """
        to check is the title match the list, return a boolean list(check_list) and the result.
    """
    
    check_list = []
    for i in titles_list:
        if db_title in i:
            check_list.append(True)
        else:
            check_list.append(False)

    result = [x for x in check_list if x == True]
    return result, check_list

In [45]:
# function test 2

landing_site = 'http://www.nasdaq.com/symbol/' + c['symbol'] + '/news-headlines'

re = get_connection(landing_site)

html = re.content

'''convert html to BeautifulSoup object'''
soup = BeautifulSoup(html , 'lxml')


'''get list of all links on webpage'''
links = soup.find_all(target="_self")

times_ = soup.find_all('small')

times_ = times_[1:]

ca = []
for i in range(len(times_)):
    ca.append(times_[i].getText())
    

for i in ca:
    c = i.strip().split(sep=" - ")[0]
    print(dateutil.parser.parse(c).strftime('%Y-%m-%d %H:%M:%S'))

titles = [title.getText() for title in links]
titles = titles[1:]

print(titles)


Success!
New Tor connection processed
New Ip Address 178.17.166.149
2019-02-01 00:04:00
2019-01-31 22:27:00
2019-01-31 22:03:26
2019-01-31 22:03:00
2019-01-31 21:51:00
2019-01-31 21:37:46
2019-01-31 21:37:07
2019-01-31 21:30:00
2019-01-31 21:30:00
2019-01-31 21:00:00
2018-12-04 16:25:00
2018-11-21 16:39:00
2018-11-20 12:40:00
2018-11-20 12:10:00
2018-11-20 12:01:00
2018-11-19 18:08:00
['Buy Facebook (FB) Stock After Q4 Earnings On Instagram, Mobile Ad Strength?', 'Nikkei rises to highest since mid-Dec, but Nomura, Nintendo drag', 'Stock Market Today: Facebook Grows Users, Tesla Earns a Profit', 'Sensex, Nifty Seen Higher At Open; Brace For Volatility', "Singapore's Grab tempts headquarters curse", 'Nasdaq Closes Out Best Month Since 2011', 'Close Update: Stocks Extend Fed-Fueled Rally, End January With Sizable Gains', 'Asian shares off 4-month high as China data disappoints', 'Thai Stock Market May Spin Its Wheels On Friday', 'Indonesia Stock Market Has Flat Lead For Friday']


In [ ]:
# function test
page = 1
landing_site = 'http://www.nasdaq.com/symbol/' + c['symbol'] + '/news-headlines'

print(landing_site)

t, n = get_news_urls(landing_site)
print(n)
for i in t:
    print(i)
print(len(n))
print(len(t))
print(c['title'])
print(c['symbol'])
checking, c_list = check_exist(c['title'], t)
print(checking)
print(c_list)

while(checking == []):
    page = page + 1
    t, n = get_news_urls(landing_site + '?page=' + str(page))
    print(n)
    for i in t:
        print(i)
    print(len(n))
    print(len(t))
    print(c['title'])
    print(c['symbol'])
    checking, c_list = check_exist(c['title'], t)
    print(checking)
    print(c_list)
    

http://www.nasdaq.com/symbol/fb/news-headlines
http://www.nasdaq.com/symbol/fb/news-headlines
Success!
New Tor connection processed
New Ip Address 217.170.197.83
['https://www.nasdaq.com/article/buy-facebook-fb-stock-after-q4-earnings-on-instagram-mobile-ad-strength-cm1091441', 'https://www.nasdaq.com/article/stock-market-today-facebook-grows-users-tesla-earns-a-profit-cm1091305', 'https://www.nasdaq.com/article/sensex-nifty-seen-higher-at-open-brace-for-volatility-20190131-01464', 'https://www.nasdaq.com/article/singapores-grab-tempts-headquarters-curse-20190131-01467', 'https://www.nasdaq.com/article/nasdaq-closes-out-best-month-since-2011-cm1091245', 'https://www.nasdaq.com/article/close-update-stocks-extend-fed-fueled-rally-end-january-with-sizable-gains-cm1091253', 'https://www.nasdaq.com/article/asian-shares-off-4month-high-as-china-data-disappoints-20190131-01460', 'https://www.nasdaq.com/article/thai-stock-market-may-spin-its-wheels-on-friday-20190131-01457', 'https://www.nasda

New Ip Address 185.4.132.183
Reconnect1
Success!
New Tor connection processed
New Ip Address 144.217.90.68
Reconnect2
Success!
New Tor connection processed
New Ip Address 89.234.157.254
Reconnect3
Success!
New Tor connection processed
New Ip Address 185.220.100.252
['https://www.nasdaq.com/article/facebook-profit-beats-wall-st-shares-jump-after-hours-20190130-01246', 'https://www.nasdaq.com/article/facebook-quarterly-profit-revenue-beat-estimates-20190130-01157', 'https://www.nasdaq.com/article/big-thumbsup-from-wall-st-after-fed-signals-patience-on-rates-20190130-01031', 'https://www.nasdaq.com/article/us-stocksbig-thumbsup-from-wall-st-after-fed-signals-patience-on-rates-20190130-01022', 'https://www.nasdaq.com/article/big-thumbs-up-from-wall-st-after-fed-signals-patience-on-rates-20190130-00995', 'https://www.nasdaq.com/article/us-stocksbig-thumbs-up-from-wall-st-after-fed-signals-patience-on-rates-20190130-00993', 'https://www.nasdaq.com/article/wall-st-climbs-on-apple-boeing-resul

New Ip Address 185.220.100.252
['https://www.nasdaq.com/article/earnings-reaction-history-facebook-inc-182-follow-through-indicator-51-sensitive-cm1090148', 'https://www.nasdaq.com/article/sony-and-facebook-are-dominating-this-next-gen-hardware-market-cm1090137', 'https://www.nasdaq.com/article/after-hours-earnings-report-for-january-30-2019-msft-fb-v-pypl-mdlz-qcom-tsla-now-ess-amp-holx-wynn-cm1090113', 'https://www.nasdaq.com/article/5-machine-learning-stocks-to-buy-for-a-smarter-portfolio-cm1090207', 'https://www.nasdaq.com/article/facebook-earnings-beat-street-shares-up-7-20190130-01167', 'https://www.nasdaq.com/article/is-facebook-stock-a-screaming-buy-ahead-of-earnings-cm1090151', 'https://www.nasdaq.com/article/facebook-inc-q4-income-rises-20190130-01159', 'https://www.nasdaq.com/article/feds-message-of-patience-extends-early-rally-on-wall-street--us-commentary-20190130-01155', 'https://www.nasdaq.com/article/facebook-keeps-attracting-digital-ads-shares-jump-after-hours-20190130

New Ip Address 197.231.221.211
['https://www.nasdaq.com/article/wall-street-wavers-as-apple-tees-up-its-quarterly-report-20190129-01182', 'https://www.nasdaq.com/article/bolsa-euasp-500-cai-pressionado-por-setor-de-tecnologia-3m-impulsiona-dow-20190129-01135', 'https://www.nasdaq.com/article/apple-aapl-in-trouble-after-facetime-bug-reveals-user-data-cm1089063', 'https://www.nasdaq.com/article/the-zacks-analyst-blog-highlights-apple-facebook-microsoft-tesla-and-boeing-cm1089170', 'https://www.nasdaq.com/article/tech-stocks-earnings-lineup-for-jan-30-chkp-fb-amp-more-cm1088923', 'https://www.nasdaq.com/article/stock-market-news-for-jan-29-2019-cm1088931', 'https://www.nasdaq.com/article/gold-hits-eightmonth-high-stocks-mixed-amid-trade-caution-results-20190129-01000', 'https://www.nasdaq.com/article/implied-volatility-surging-for-facebook-fb-stock-options-cm1088943', 'https://www.nasdaq.com/article/will-it-get-better-for-faang-stocks-this-earnings-season-cm1088950', 'https://www.nasdaq.c

In [4]:
# to close the database connection 
c.close

<function Cursor.close>

In [ ]:
# drop table command
c.execute("DROP Table nasdaq_stock_news")